In [7]:
import torch
import torch.nn as nn
import random
import cv2
import numpy as np
import sys
sys.path.insert(0, "/home/nguyen.thanh.huyenb/yolov7/yolov7_train")

In [39]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
device

device(type='cpu')

In [43]:
weights = "/home/nguyen.thanh.huyenb/yolov7/yolov7_train/runs/train/yolov7-qat-/weights/best.pt"
model = torch.load(weights)['model'].float().fuse().eval()

from models.common import Conv

for m in model.modules():
    if type(m) in [nn.Hardswish, nn.LeakyReLU, nn.ReLU, nn.ReLU6, nn.SiLU]:
        m.inplace = True  # pytorch 1.7.0 compatibility
    elif type(m) is nn.Upsample:
        m.recompute_scale_factor = None  # torch 1.11.0 compatibility
    elif type(m) is Conv:
        m._non_persistent_buffers_set = set()  # pytorch 1.6.0 compatibility

model.to(device)

Fusing layers... 
RepConv.fuse_repvgg_block
RepConv.fuse_repvgg_block
RepConv.fuse_repvgg_block
IDetect.fuse


Model(
  (model): Sequential(
    (0): Conv(
      (conv): Conv2d(3, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (act): SiLU(inplace=True)
    )
    (1): Conv(
      (conv): Conv2d(32, 64, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
      (act): SiLU(inplace=True)
    )
    (2): Conv(
      (conv): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (act): SiLU(inplace=True)
    )
    (3): Conv(
      (conv): Conv2d(64, 128, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
      (act): SiLU(inplace=True)
    )
    (4): Conv(
      (conv): Conv2d(128, 64, kernel_size=(1, 1), stride=(1, 1))
      (act): SiLU(inplace=True)
    )
    (5): Conv(
      (conv): Conv2d(128, 64, kernel_size=(1, 1), stride=(1, 1))
      (act): SiLU(inplace=True)
    )
    (6): Conv(
      (conv): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (act): SiLU(inplace=True)
    )
    (7): Conv(
      (conv): Conv2d(64, 64, kernel_size=(3, 3), strid

In [44]:
stride = int(model.stride.max()) 
stride

32

In [45]:
names = model.module.names if hasattr(model, 'module') else model.names
colors = [[random.randint(0, 255) for _ in range(3)] for _ in names]
names, colors

(['bicycle', 'car', 'motorcycle', 'airplane', 'bus', 'train', 'truck', 'boat'],
 [[186, 240, 13],
  [177, 20, 91],
  [136, 250, 24],
  [117, 97, 245],
  [235, 240, 16],
  [51, 205, 23],
  [108, 161, 64],
  [165, 51, 252]])

## Load image

In [46]:
from utils.datasets import letterbox
from utils.general import scale_coords, non_max_suppression

In [66]:
image_path = "/home/nguyen.thanh.huyenb/yolov7/training_minicoco_dataset/test/000000001722.jpg"
image_path = "/home/nguyen.thanh.huyenb/yolov7/training_minicoco_dataset/test/images/000000237309.jpg"
image0 = cv2.imread(image_path)
image0.shape
image0[0]

array([[240, 217, 201],
       [241, 218, 202],
       [240, 220, 203],
       ...,
       [243, 223, 206],
       [243, 223, 206],
       [244, 224, 206]], dtype=uint8)

In [68]:
image = letterbox(image0, 320, stride=stride)[0]
image.shape

(256, 320, 3)

In [69]:
image = image[:, :, ::-1].transpose(2, 0, 1)  # BGR to RGB, to 3x416x416
image = np.ascontiguousarray(image)
image.shape

(3, 256, 320)

In [70]:
image = torch.from_numpy(image).to(device).float()
image /= 255.0
image = image.unsqueeze(0)

In [71]:
with torch.no_grad():   # Calculating gradients would cause a GPU memory leak
    pred = model(image)[0]
pred.shape

/home/nguyen.thanh.huyenb/miniconda3/envs/yolo/lib/python3.10/site-packages/torch/functional.py:504: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at /opt/conda/conda-bld/pytorch_1666642922335/work/aten/src/ATen/native/TensorShape.cpp:3190.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]


torch.Size([1, 5040, 13])

In [72]:
pred = non_max_suppression(pred)
pred

[tensor([[ 71.61380,  51.01521, 245.63321, 219.82658,   0.94192,   4.00000],
         [ 29.85685, 143.07176,  75.19416, 188.02525,   0.57919,   6.00000],
         [ 30.14836, 142.77089,  74.00276, 187.96458,   0.50369,   4.00000],
         [  8.26203, 149.24913,  32.22172, 180.46855,   0.42121,   6.00000]])]

In [73]:
det = pred[0]
det[:, :4] = scale_coords(image.shape[2:], det[:, :4], image0.shape).round()
det

tensor([[1.43000e+02, 8.60000e+01, 4.91000e+02, 4.24000e+02, 9.41919e-01, 4.00000e+00],
        [6.00000e+01, 2.70000e+02, 1.50000e+02, 3.60000e+02, 5.79185e-01, 6.00000e+00],
        [6.00000e+01, 2.70000e+02, 1.48000e+02, 3.60000e+02, 5.03691e-01, 4.00000e+00],
        [1.70000e+01, 2.82000e+02, 6.40000e+01, 3.45000e+02, 4.21209e-01, 6.00000e+00]])

In [74]:
for *xyxy, conf, cls in reversed(det):
    # print(xyxy)
    label = f'{names[int(cls)]} {conf:.2f}'
    print(label)

truck 0.42
bus 0.50
truck 0.58
bus 0.94
